<a href="https://colab.research.google.com/github/anadeleame/bioinfo-postgraduation/blob/main/Libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fake.fasta to fake.fasta
Saving fake.vcf to fake.vcf


1 - Open the fasta file and print all of the reads names in the file.

In [ ]:
fastafile = open('fake.fasta', 'r')

for line in fastafile:
  if '>' in line:
    print(line)

2 - How many variants were called in the VCF file?


In [ ]:
import re                                              #1step: importing the libraries I need
from Bio import Seq

with open('fake.vcf', 'r') as vcf:                     #2step: opening fake.vcf as a readable file
  with open('variants.txt','w') as justvariants:       #3step: creating a new file named variants.txt as a writable file
    for line in vcf:                                   #4step: reading fake.vcf using loop
      if not re.search('#', line):                     #5step: excluding headers from vcf file
        justvariants.writelines(line)                  #6step: writing lines in the variants.txt without headers

counter = 0                                            #7step: creating a counter for count variants

with open('variants.txt', 'r') as justvariants:        #8step: opening variants.txt as a readable file
  for line in justvariants:                            #9step: reading variants.txt using loop
    counter = counter + line.count("ExcessHet=")       #10step: counting lines that countains the string "ExcessHet=" as an indirect way to count variants. I tried counting with 'chr' but it was counting more variants than it actually exists (?)

print(f'It were called {counter} variants')           #11step: printing how many variantes were found

Foram chamadas 9947 variantes


3- Using the system you want, print the first 20 variants that have quality call > 1000.


In [ ]:
import re                                                   #1step: import the libraries you need to use

n = 0                                                       #2step: I just want to read the twenty first variants and not the whole file, so this 'n' needs to finish at 19
with open('fake.vcf', 'r') as archive_vcf:                  #3step: opening fake.vcf as a readable file
  with open('variants.txt','r') as justvariants:            #4step: opening variants.txt as a readable file
    for lines in justvariants:                              #5step: reading variants.txt using loop
      if float(lines.split('\t')[5]) > 1000 and n < 20:     #6step: for every line in variants.txt, cut the fifth column (quality data) IF the value is greater than 1000 (I needed to convert it to float because it was a string) and 'n' is lesser than 20
        print(lines)                                        #7step: if the condition is accepted, print the value in the fifth column
        n = n + 1                                           #8step: if the condition is accepted, sum +1 to the last value of 'n'

4 - In a research laboratory, one seeks to discover a bacterial genome. The sequencing was performed by Illumina and the assembly by an X assembler. However, this assembly came out very fragmented. The bioinformatician, with his knowledge, was able to identify, through a gene search, a fragment that would correspond to the genome in question and carried out the analysis of its GC content. Knowing that this genome has a GC content between 0.40 and 0.38, find possible fragments that belong to this organism in the file "fake.fasta" - **Challenge**

In [ ]:
from Bio.SeqUtils import gc_fraction                                                 #1step: importing the libraries I need

with open('fake.fasta', 'r') as fasta:                                               #2step: opening fake.fasta as a readable file
  with open('wlines.txt', 'w') as fasta_w:                                           #step: creating a new file named wlines.txt (without lines)
    for line in fasta:                                                               #4step: reading fake.fasta using loop
      if '>fasta1' in line:                                                          #5step: if '>fasta1' in line...
        fasta_w.writelines(line)                                                     #6step: ... just write the line
      elif '>' in line and not ">fasta1" in line:                                    #7step: if '>' is in line but it's not '>fasta1'...
        fasta_w.writelines('\n')                                                     #8step: create a new line (to separate the names of the fastas). That's why I did the if above, because if I just write this 'if' statement, it will create a file starting with a line jump
        fasta_w.writelines(line)                                                     #9step: write the line
      else:                                                                          #10step: if it's not the 'if' condition or 'elif' condition, that is, the sequence line...
        fasta_w.writelines(line.split('\n'))                                         #11step: write the lines without the jump line within the sequence

with open('wlines.txt', 'r') as fasta_r:                                             #12step: opening wlines.txt as a readable file
  for seq in fasta_r:                                                                #13step: reading wlines.txt using loop
    if not seq.startswith('>'):                                                      #14step: if the line doesn't start with '>' (headers)...
      gc_content = gc_fraction(seq)                                                  #15step: ... calculate 'GC' content
      if gc_content > 0.38 and gc_content < 0.40:                                    #16step: if the 'GC' content is between 0.38 and 0.40...
        print(f' A possible fragment that belongs to this organism is: {seq}')      #17step: print possible sequences



In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction

with open('fake.fasta', 'r') as fasta:                                                #the same resolution as above but using SeqIO.parse to concatenate the sequence (taking off jump lines) and calculating 'GC' content directly
  for seq_record in SeqIO.parse("fake.fasta", "fasta"):
    gc_content = gc_fraction(seq_record)
    if gc_content > 0.38 and gc_content < 0.40:
      print(f'A possible fragment that belongs to the organism is - {seq_record}')


5- We have a VCF containing multiple variants of a patient. However, our laboratory only analyzes variants with a minimum read coverage of 30. Print these variants to an output file and how many variants were filtered out.

In [ ]:
import re

counter = 0

with open ('variants.txt', 'r') as variants:
  with open('output_file.txt', 'w') as outputfile:
    with open('data.txt', 'w') as datas:
      with open('lastcolumn.txt', 'w') as lastcolumn:
        for line in variants:
          lastcolumn.writelines(line.split('\t')[7])
          lastcolumn.writelines('\n')

      with open('lastcolumn.txt', 'r') as lastcolumn_r:
        for data in lastcolumn_r:
          dp = re.findall(r'DP=([0-9]+).*',data)
          datas.writelines(dp)
          datas.writelines('\n')

with open ('variants.txt','r') as variants:
  with open ('data.txt', 'r') as datas_r:
    with open('output_file.txt', 'w') as outputfile:
      for linefile1, linefile2 in zip(variants, datas_r):
        dpfloat = float(linefile2)
        if dpfloat > 30.0:
          outputfile.writelines(linefile1)

with open('output_file.txt', 'r') as output_r:
  for filtered_variants in output_r:
    counter =  counter + filtered_variants.count('ExcessHet=')
print(f'{counter} variants were found')